In [ ]:
from pyprot.protein import Protein

First, let's load up some proteins.

In [ ]:
ids = ["1A0I", "1A49", "1A5U", "1A82", "1AQ2"] #,"1ASZ", "1ATN", "1ATP", "1AYL", "1B0U", "1B38"]
#ids = ["1A0I"]
proteins = [Protein.fetch(_id) for _id in ids]
proteins = list(filter(lambda p: p.df.shape[0] < 5000, proteins))

#You can also just pass a file directly:
#proteins.append(Protein("1i7l.pdb"))

We automatically have a dataframe of the atoms for each protein:

In [ ]:
proteins[0].df.head()

We might want to add some features, a target distance for example:

In [ ]:
import numpy as np
for protein in proteins:
    protein.df = protein.df[~protein.df.coord.isnull()]
    ATP_coords = protein.df[protein.df.resname == "ATP"].coord.to_list()
    print("Found {} ATP atoms".format(len(ATP_coords)))
    protein.df["distance"] = protein.df.coord.apply(
        lambda atom: min(map(lambda atp_atom: np.linalg.norm(atom-atp_atom), ATP_coords))
    )
    #protein.discard_ligands()
    # Sanity check
    protein.df = protein.df.loc[
        protein.df.apply(lambda row: row["full_id"][4][0] == "CA", axis=1),
        :].reset_index(drop=True)

And discard ligands to keep only protein atoms:

In [ ]:
for protein in proteins:
    chains_with_ligand = protein.df[protein.df.distance <= 6.0].chain.unique()
    protein.select_chains(chains_with_ligand)

Make a graph by Delauney triangulation:

In [ ]:
from pyprot.structure import Perseus
import pyprot.graph_models as graph_models

structures = []
graphs = []

for i in range(len(proteins)):
    print("Processing #{} -- shape {}".format(i, proteins[i].df.shape))
    structure = proteins[i].generate_structure(lambda row: row["full_id"][4][0] == "CA")
    perseus = Perseus()
    perseus.execute_persistent_hom(proteins[i])

    structure_model = graph_models.StructureGraphGenerator()
    proteins[i].generate_graph(structure_model,
        {"step": structure.persistent_hom_params["b3_step"]})
    # Depth features
    depths, _ = structure.calculate_depth(proteins[i].graph)
    for node_idx, depth in depths.items():
        proteins[i].graph.nodes[node_idx]["depth"] = depth

    # Rest of features
    structure_model.add_features(proteins[i].df, columns = [
        "bfactor", "resname", "x", "y", "z", "distance"
    ])
    graphs.append(structure_model)
    structures.append(structure)

In [ ]:
# Plot the structure of a protein
proteins[0].structure.plot(proteins[0].structure.persistent_hom_params["b3_step"], 1)

We can turn a graph into a dataframe by propagating features along neighbors:

In [ ]:
dfs = [
    graph_models.GraphModel.graph_to_dataframe(
        graph_models.GraphModel.get_diffused_graph(
            p.graph, steps=2, keys=["depth", "bfactor"]))
    for p in proteins]

In [ ]:
dfs[0].head()

Aaaand we can also make folds using sequence similarity:

In [ ]:
from pyprot.groupfolds import CDHitGroup

groups = CDHitGroup.get_group(proteins)
groups

So now we can run a simple model for binding site prediction:

In [ ]:
import sklearn.model_selection
import sklearn.metrics
from sklearn import linear_model
import sklearn.preprocessing
import pandas


for i, df in enumerate(dfs):
    df["group"] = groups[i]

dataset = pandas.concat(dfs)
row_groups = dataset.group
row_target = dataset.distance

groupk = sklearn.model_selection.GroupKFold(n_splits=2)
dataset = dataset.drop(["distance", "group", "full_id"], axis=1)

dataset.head()

In [ ]:
enc = sklearn.preprocessing.OneHotEncoder()
onehot_res = enc.fit_transform(dataset.resname.to_numpy().reshape(-1, 1))

X = np.concatenate([dataset.drop("resname", axis=1).to_numpy(), onehot_res.todense()], axis=1)

row_target = row_target.to_numpy()
row_groups = row_groups.to_numpy()

In [ ]:
def touches_ligand(x):
    """Make a soft class boundary by smoothing the distribution"""
    return int(x <= 4 or (x<=6 and np.random.binomial(1, 1-(x-4)/2) == 1))

vec_touches = np.vectorize(touches_ligand)

In [ ]:
row_target = vec_touches(row_target)

In [ ]:
for train_index, test_index in groupk.split(X, row_target, row_groups):
    reg = linear_model.Lasso(alpha=0.1)
    reg.fit(X[train_index], row_target[train_index])
    y_hat = reg.predict(X[test_index])
    print(sklearn.metrics.roc_auc_score(row_target[test_index], y_hat))